In [41]:
from __future__ import division
import sys
import vcf
from collections import Counter
import pprint as pp
from pydoc import help
import matplotlib.pyplot as plt

In [50]:

sex_info = "/home/djeffrie/Data/Ribe_LM/pop_map_clutch.txt"

In [21]:
vcf_path = "/home/djeffrie/Data/Ribe_LM/batch_1.vcf"
alteredvcfpath = "%s%s" % (vcf_path, ".altered")

    oldvcf = open(vcf_path, 'r').readlines()
    alteredvcf = open(alteredvcfpath, 'w')    
    
    for line in oldvcf:
        if "Allele Depth" not in line:
            alteredvcf.write(line)
        elif "Allele Depth" in line:
            line = '##FORMAT=<ID=AD,Number=.,Type=Integer,Description="Allele Depth">\n'
            alteredvcf.write(line)
    alteredvcf.close()
    
    altered_vcf = open(alteredvcfpath, 'r')
    
    myvcf = vcf.Reader(altered_vcf)

In [52]:
sex_file = open(sex_info, 'r').readlines()
offspring = []

for line in sex_file: ## Note, only samples in the sex_info file are used
        
    sample_name = line.split()[0]
    sex = line.split()[1]                            
    if sex == "F" or sex == "f":
        Mother = sample_name
            
    elif sex == "M" or sex == "m":
        Father = sample_name
        
    elif sex == "O" or sex == "o":
        offspring.append(sample_name)                
offspring = sorted(offspring)
print offspring

['T1', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T2', 'T20', 'T21', 'T22', 'T23', 'T24', 'T25', 'T26', 'T27', 'T28', 'T29', 'T3', 'T30', 'T31', 'T32', 'T33', 'T34', 'T35', 'T36', 'T37', 'T38', 'T39', 'T4', 'T40', 'T41', 'T42', 'T43', 'T44', 'T45', 'T46', 'T47', 'T48', 'T49', 'T5', 'T50', 'T51', 'T52', 'T53', 'T54', 'T55', 'T56', 'T57', 'T58', 'T59', 'T6', 'T60', 'T61', 'T62', 'T63', 'T64', 'T65', 'T66', 'T67', 'T68', 'T69', 'T7', 'T70', 'T71', 'T72', 'T73', 'T74', 'T75', 'T76', 'T77', 'T78', 'T79', 'T8', 'T80', 'T9']


In [68]:
vcf_file = open("/home/djeffrie/Data/Ribe_LM/batch_1.vcf.altered", 'r')
myvcf = vcf.Reader(vcf_file)
locus_counter = 0
Par_hom_4_diff_alleles = 0
Par_hom_4_same_alleles = 0
Loc_offsp_hets_perc_Par_diff_alleles = []

for record in myvcf:
        locus_counter += 1
        Mat_gt_bases = None
        Pat_gt_bases = None
        
        Sample_headers = []
        sample_names = []
        Parental_alleles = []
        offspring_gt_types = []
        offspring_gt_types_dict = {}
        offspring_gt_bases_dict = {}
        allele_dropout_scores = {}
        
    
        Loc_Id = "%s_%s" % (record.ID, record.POS)
        
        for sample in record.samples:
            
            name = sample.sample
            genotype = sample['GT']
            allele_dropout_scores[record] = {}
            
            sample_names.append(name)
            
            ## Get Mother and Father information
            
            if name == Mother:
                print "Mother", sample.gt_type, sample.gt_bases
                if not sample.called == False:
                    Mat_gt_bases = sample.gt_bases
                    if sample.gt_type == 0 or sample.gt_type == 2:
                        mat_het = False
                    elif sample.gt_type == 1:
                        mat_het = True   
                    else:
                        print "something else"
                elif sample.called == False:
                    mat_het = None

                    
            elif name == Father:
                print "Father", sample.gt_type, sample.gt_bases
                if not sample.called == False:
                    Pat_gt_bases = sample.gt_bases
                    if sample.gt_type == 0 or sample.gt_type == 2:
                        pat_het = False
                    elif sample.gt_type == 1:
                        pat_het = True
                    else:
                        print "something else"
                elif sample.called == False:
                    pat_het = None
        
        ## For loci where Parents are homozygous for different alleles:
        
        print pat_het, mat_het
        hets = []
        
        if pat_het == False and mat_het == False: ## If both parents are homozygous
            print "Parents homozygous for diff alleles"
            if not Mat_gt_bases == Pat_gt_bases:
                print Mat_gt_bases, Pat_gt_bases
                Par_hom_4_diff_alleles += 1
        
                for sample in record.samples: ## Loop through offspring and get heterozygosity info      
                    name = sample.sample
                    if name in offspring:
                        if sample.gt_type == 1:
                            het = True
                        elif sample.gt_type == None:
                            het = None
                        else:
                            het = False
                        hets.append(het)
                counted = Counter(hets)
                total_called = (counted[False]+counted[True])
                Perc_hom_offspr = counted[False]/total_called
                Perc_het_offspr = counted[True]/total_called
                Loc_offsp_hets_perc_Par_diff_alleles.append(Perc_het_offspr)
                #print Perc_hom_offspr
                #print Perc_het_offspr
            
            elif Mat_gt_bases == Pat_gt_bases:
                print "Parents homozygous for same alleles"
                
                Par_hom_4_same_alleles += 1
        
                for sample in record.samples: ## Loop through offspring and get heterozygosity info      
                    name = sample.sample
                    if name in offspring:
                        if sample.gt_type == 1:
                            het = True
                        elif sample.gt_type == None:
                            het = None
                        else:
                            het = False
                        hets.append(het)
                counted = Counter(hets)
                total_called = (counted[False]+counted[True])
                Perc_hom_offspr = counted[False]/total_called
                Perc_het_offspr = counted[True]/total_called
                #Loc_offsp_hets_perc.append(Perc_het_offspr)
                print Perc_hom_offspr
                print Perc_het_offspr
            
                

if len(Loc_offsp_hets_perc_Par_diff_alleles) > 0:
    plt.hist(Loc_offsp_hets_perc_Par_diff_alleles, bins = 20)
    plt.show()
                

Father 0 T/T
Mother 1 C/T
False True
Father 0 C/C
Mother 1 C/G
False True
Father 0 C/C
Mother 1 A/C
False True
Father 1 A/G
Mother 0 G/G
True False
Father 1 C/T
Mother 1 C/T
True True
Father 1 C/T
Mother 0 T/T
True False
Father 1 A/T
Mother 0 T/T
True False
Father 1 A/T
Mother 0 T/T
True False
Father 0 C/C
Mother 1 C/G
False True
Father 1 G/T
Mother 0 T/T
True False
Father 0 T/T
Mother 1 C/T
False True
Father 0 A/A
Mother 1 A/C
False True
Father 1 C/T
Mother 0 C/C
True False
Father 1 C/T
Mother 0 C/C
True False
Father 1 A/C
Mother 0 A/A
True False
Father 1 A/C
Mother 0 C/C
True False
Father 1 C/T
Mother 1 C/T
True True
Father 1 A/T
Mother 1 A/T
True True
Father 0 T/T
Mother 1 C/T
False True
Father 0 C/C
Mother 1 C/T
False True
Father 0 G/G
Mother 1 G/T
False True
Father 1 C/T
Mother 1 C/T
True True
Father 1 A/G
Mother 1 A/G
True True
Father 1 A/C
Mother 0 A/A
True False
Father 0 A/A
Mother 1 A/G
False True
Father 1 C/G
Mother 0 G/G
True False
Father 1 A/G
Mother 1 A/G
True True
Father 

In [34]:
help(py.hist)

Help on function hist in module matplotlib.pyplot:

hist(x, bins=10, range=None, normed=False, weights=None, cumulative=False, bottom=None, histtype=u'bar', align=u'mid', orientation=u'vertical', rwidth=None, log=False, color=None, label=None, stacked=False, hold=None, **kwargs)
    Plot a histogram.
    
    Compute and draw the histogram of *x*. The return value is a
    tuple (*n*, *bins*, *patches*) or ([*n0*, *n1*, ...], *bins*,
    [*patches0*, *patches1*,...]) if the input contains multiple
    data.
    
    Multiple data can be provided via *x* as a list of datasets
    of potentially different length ([*x0*, *x1*, ...]), or as
    a 2-D ndarray in which each column is a dataset.  Note that
    the ndarray form is transposed relative to the list form.
    
    Masked arrays are not supported at present.
    
    Parameters
    ----------
    x : (n,) array or sequence of (n,) arrays
        Input values, this takes either a single array or a sequency of
        arrays which ar